## turbo emittance optimization

In [ ]:
# optionally add scripts location to path
if True:
    import sys
    sys.path.append("../../")
    sys.path.append("../")

run_dir = "/home/physics3/ml_tuning/20230818_LCLS_optimize/"
print(sys.path)


In [ ]:
from scripts.image import ImageDiagnostic
import yaml

fname = "../OTR3_config.yml"
image_diagnostic = ImageDiagnostic.parse_obj(yaml.safe_load(open(fname)))
image_diagnostic.save_image_location = run_dir
image_diagnostic.n_fitting_restarts = 2
image_diagnostic.visualize = False
image_diagnostic.background_file = run_dir + "OTRS_IN20_621_background.npy"
print(image_diagnostic.yaml())

In [ ]:
from scripts.automatic_emittance import ScreenEmittanceMeasurement, BeamlineConfig

drift_length = 2.2
beamline_config = BeamlineConfig(
    scan_quad_pv="QUAD:IN20:525:BCTRL",
    scan_quad_range=[-2.0, 2.0],
    scan_quad_length=0.1068,
    transport_matrix_x =[[2.29099365,    13.24790498],
        [ -0.20459861,    -0.74662055]],
    transport_matrix_y = [[0.82340723,   -10.58404798],
        [0.04426516,     0.64548308]],
    beam_energy=0.135,
)


measurement = ScreenEmittanceMeasurement(
    image_diagnostic=image_diagnostic,
    beamline_config=beamline_config,
    run_dir=run_dir,
    visualize=True
)

measurement.dump_yaml()

In [ ]:
def evaluate_emittance(inputs):
    global measurement
    # set PVs
    for k, v in inputs.items():
        print(f'CAPUT {k} {v}')
        caput(k, v)

    sleep(2.0)

    # get beam sizes from image diagnostic
    metadata = inputs
    results, emit_xopt = measurement.run()
